# Clip OSMFile using OSMOSIS

This method uses OSMOSIS to get an OSM extract from a big OSM File

### Parameters: ###

**INTEREST_AREA** - Extent of your interest area. Could be a path to a file (e.g. ESRI Shapefile or KML), a wkt string, or a list with wkt strings:

```Python
# If it is WKT
INTEREST_AREA = "POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))"

# OR if it is a List of WKT
# The list order should be: left; right; bottom; top
INTEREST_AREA = ["POLYGON ((30 10, 40 40, 20 40, 10 20, 30 10))", "POLYGON ((15 5, 20 20, 10 20, 5 10, 15 5))"]

# OR if it is a File
INTEREST_AREA = '/path/to/yourboundary.shp'
```

**OSMDATA** - Path to the output file that will have OSM Data available in the Interest Area;

**EPSG** - EPSG Code of the Spatial Reference System of INTEREST_AREA.

In [1]:
from gasp.gt.gop.osm.ovly import osm_extraction

INTEREST_AREA = '/mnt/d/gwork/tstgsrv/lmt_flainar.shp'
OSM_FILE      = '/mnt/d/gwork/osm2lulc_imgcls/osmdata/portugal-latest.osm.pbf'
OSM_EXTRACT   = '/mnt/d/gwork/tstgsrv/osm_flainar.xml'

osm_extraction(INTEREST_AREA, OSM_FILE, OSM_EXTRACT)

'/mnt/d/gwork/tstgsrv/osm_flainar.xml'

# Convert OSM data into Feature Class #

In [ ]:
from gasp.gt.to.shp import osm_to_featcls

osm_to_featcls(
    '/home/jasp/flainar/portugal-latest.osm.pbf',
    '/home/jasp/flainar/', useXmlName=True, outepsg=32629)

### OSM File to PSQL DB ###

In [ ]:
from gasp.sql.to import osm_to_pgsql

inOsm = '/home/osmtolulc/mrgis/europe-latest.osm.pbf'

osm_to_pgsql(inOsm, 'osm_europe')

# OSM File to Relation Sctruture: #

In [ ]:
from gasp3.sql.to.osm import osm_to_relationaldb

conDB = {
    'HOST' : 'localhost', 'PORT' : '5433', 'USER' : 'postgres',
    'PASSWORD' : 'admin', 'DATABASE' : 'flainar_osm',
    'TEMPLATE' : 'postgis_template'
}

inOsm = '/home/jasp/flainar/osm_centro.xml'

inSchema = {
    "TBL" : 'points', 'FID' : 'ogc_fid',
    "COLS" : [
        'name', 'osm_id',
        "ST_X(wkb_geometry) AS longitude",
        "ST_Y(wkb_geometry) AS latitude",
        "wkb_geometry AS geom",
        "NULL AS featurecategoryid",
        "NULL AS flainarcategoryid",
        "NULL AS createdby",
        "NOW() AS createdon",
        "NULL AS updatedon",
        "NULL AS deletedon"
    ],
    "NOT_KEYS" : [
        'ogc_fid', 'osm_id', 'name', "wkb_geometry",
        'healthcare2', 'other_tags'
    ]
}

osmGeoTbl = {"TBL" : 'position', "FID" : 'positionid'}

osmCatTbl = {
    "TBL" : 'osmcategory', "FID" : "osmcategoryid",
    "KEY_COL" : "keycategory", "VAL_COL" : "value",
    "COLS" : [
        "NULL AS createdby", "NOW() AS createdon",
        "NULL AS updatedon", "NULL AS deletedon"
    ]
}

osmRelTbl = {
    "TBL" : "position_osmcat", "FID" : 'pososmcatid'
}

outsql = '/home/jasp/flainar/flainar_osm.sql'
osm_to_relationaldb(conDB, inOsm, inSchema, osmGeoTbl, osmCatTbl, osmRelTbl, outSQL=outsql)